# Baseline Traduccion Español 
Usando la descripcion de las slides

## Install and data

In [1]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Rutas de los archivos zip en Google Drive
test_zip_path = '/content/drive/MyDrive/test.zip'
test_lst = '/content/drive/MyDrive/test.lst'

# Directorio de destino en Colab
colab_content_path = '/content/'

# Copiar los archivos zip a Colab
!cp "{test_zip_path}" "{colab_content_path}"
!cp "{test_lst}" "{colab_content_path}"

print(f"Archivos copiados a {colab_content_path}")
print(os.listdir(colab_content_path))

Mounted at /content/drive
Archivos copiados a /content/
['.config', 'test.lst', 'drive', 'test.zip', 'sample_data']


In [2]:
!unzip /content/test.zip -d /content/test

Archive:  /content/test.zip
   creating: /content/test/404/
   creating: /content/test/545006/
   creating: /content/test/596001/
   creating: /content/test/605000/
   creating: /content/test/606/
  inflating: /content/test/test.lst  
   creating: /content/test/606/manual_transcription/
   creating: /content/test/606/manual_translations/
  inflating: /content/test/606/slides.pptx  
  inflating: /content/test/606/606.m4a  
   creating: /content/test/606/manual_transcription/sentence_segmented/
  inflating: /content/test/606/manual_transcription/ie606.srt  
  inflating: /content/test/606/manual_transcription/ie606.txt  
  inflating: /content/test/606/manual_transcription/sentence_segmented/ie606.srt  
   creating: /content/test/606/manual_translations/de/
   creating: /content/test/606/manual_translations/es/
   creating: /content/test/606/manual_translations/fr/
   creating: /content/test/606/manual_translations/sl/
  inflating: /content/test/606/manual_translations/fr/606.lst  
  infla

In [3]:
! pip install  backoff  transformers==4.49
! pip install sacrebleu unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 81.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [9]:
# load dict
import pickle
with open('/content/drive/MyDrive/images_descriptions.pkl', 'rb') as f:
    DESCRIPTIONS = pickle.load(f)
DESCRIPTIONS

{'404': ['1) Título/tema: La diapositiva se titula "Welcome!" y se trata de una sesión de aprendizaje en línea que comienza a las 18:15 horas CET.\n2) 2–3 ideas clave: La diapositiva informa que la sesión es CME acreditada y que al final de la presentación, los participantes serán dirigidos a la evaluación y prueba de opción múltiple.\n3) Elementos visuales: La diapositiva contiene un logotipo en la esquina superior derecha, un cuadro de diálogo en la parte inferior con opciones de "Yes" y "No" para unirse a la conferencia, y un hashtag "#e_ESO" en la parte inferior.',
  '1) Título/tema: La diapositiva se titula "e-sesion 404" y discute el impacto de la cirugía oncológica en los resultados.\n2) 2–3 ideas clave: La diapositiva presenta a dos expertos, Prof. Charles M. Balch y Prof. Riccardo Audisio, que discuten este tema. También menciona que el contenido de la diapositiva se extrajo de la política de e-ESO, que proporciona nuevas habilidades y conocimientos para oncólogos y otros médi

## Load

### Load Test data

In [ ]:
from pathlib import Path
import subprocess, soundfile as sf, datetime as dt

FFMPEG = "ffmpeg"          # o la ruta absoluta si no está en PATH

def m4a_to_wav(path_m4a, sr_out=16_000):
    """Convierte 1 × .m4a → .wav (mono, 16 kHz) y devuelve la ruta del WAV."""
    path_m4a = Path(path_m4a)
    wav_path = path_m4a.with_suffix(".wav")
    subprocess.run(
        [FFMPEG, "-loglevel", "error", "-y", "-i", str(path_m4a),
         "-ac", "1", "-ar", str(sr_out), str(wav_path)],
        check=True
    )
    return wav_path



def cut_segments(wav_path, transcripciones, sr=16_000):
    """
    Extrae los fragmentos indicados por .start / .end de cada objeto `seg`
    (timedelta, str o float s) y devuelve una lista de np.ndarray.
    """
    clips = []
    with sf.SoundFile(str(wav_path)) as f:
        if f.samplerate != sr:
            raise ValueError(f"El WAV está a {f.samplerate} Hz, no a {sr}")
        for seg in transcripciones:
            start_s = float(seg.split()[0])
            end_s   = float(seg.split()[1])
            f.seek(int(start_s * sr))
            frames  = int((end_s - start_s) * sr)
            clips.append(f.read(frames, dtype="float32"))
    return clips, sr                             # sr devuelto solo una vez


In [6]:
from typing import List, Optional
def get_files(folder='test'):
  with open(f'/content/{folder}/{folder}.lst', 'r', encoding='utf-8') as f:
    files = f.read()
    files = files.splitlines()
    print(files)
    return files

def slides_in_interval(
    start: float,
    end: float,
    intervals: List[dict],
    *,
    min_fraction: float = 0.10,   # % mínimo del segmento que debe cubrir la slide
    min_seconds: float = 0.0,     # y/o segundos absolutos mínimos de cobertura
    slack: float = 0.25,          # margen (±s) para tolerar pequeñas imprecisiones
    sort_by_coverage: bool = True,
    top_k: Optional[int] = 1
) -> List[int]:
    """
    Devuelve las slides que realmente 'cuentan' en [start, end), filtrando
    las que cubren menos de min_fraction del segmento o menos de min_seconds.

    - 'slack' expande el segmento a [start-slack, end+slack] para evitar cortar
      por milésimas (timings manuales, FPS, etc.).
    - Si sort_by_coverage=True, ordena por mayor cobertura; si no, por aparición.
    - 'top_k' limita el nº de slides devueltas (tras el filtro).
    """
    seg_len = max(1e-6, end - start)  # evita división por cero
    seg_lo = start - slack
    seg_hi = end + slack

    cover_by_slide = {}   # slide_id -> (cover_seconds, first_order)

    order = 0
    for seg in intervals:
        slide = seg["slide"]
        if slide is None:
            continue

        s_lo = seg["start"]
        s_hi = float("inf") if seg["end"] is None else seg["end"]

        # intersección con margen
        lo = max(seg_lo, s_lo)
        hi = min(seg_hi, s_hi)
        overlap = max(0.0, hi - lo)
        if overlap <= 0:
            continue

        if slide not in cover_by_slide:
            cover_by_slide[slide] = [0.0, order]
            order += 1
        cover_by_slide[slide][0] += overlap

    # Filtro por cobertura
    out = []
    for slide, (cover_s, first_order) in cover_by_slide.items():
        frac = cover_s / seg_len
        if cover_s >= min_seconds or frac >= min_fraction:
            out.append((slide, cover_s, first_order))

    if not out:
        return []

    # Orden
    if sort_by_coverage:
        out.sort(key=lambda x: (-x[1], x[2]))   # más cobertura primero
    else:
        out.sort(key=lambda x: x[2])            # por aparición

    if top_k is not None:
        out = out[:top_k]

    return [slide for slide, _, _ in out]



In [7]:
from pathlib import Path
from typing import List, Dict, Tuple, Optional

def _ms_to_seconds(hms: str) -> float:
    m, s, _ = hms.split(":")
    return int(m)*60 + int(s)

def parse_slide_blocks(path: str | Path) -> Dict[str, Dict]:
    """
    Lee un archivo con múltiples bloques:
      <ID> <URL>
      <slide_id|None> <HH:MM:SS>
      ...
    Devuelve dict: { id: {"url": str, "timeline": [(t_sec, slide_id|None), ...]} }
    """
    p = Path(path)
    blocks: Dict[str, Dict] = {}
    cur_id = None
    for line in p.read_text().splitlines():
        line = line.strip()
        if not line:
            continue

        parts = line.split()
        # Cabecera: "<ID> <URL>"
        if len(parts) == 2 and parts[1].startswith("http"):
            cur_id = parts[0]
            blocks[cur_id] = []
            continue

        # Línea de cambio: "<slide_id|None> <HH:MM:SS>"
        if cur_id is not None and len(parts) == 2 and ":" in parts[1]:
            slide_raw, hms = parts
            slide_id = None if slide_raw == "None" else int(slide_raw)
            t = _ms_to_seconds(hms)
            blocks[cur_id].append((t, slide_id))

    # Ordena por tiempo y asegura que hay un evento en t=0
    for k,v in blocks.items():
        v.sort(key=lambda x: x[0])

    return blocks

def build_slide_intervals(timeline: List[Tuple[float, Optional[int]]]) -> List[Dict]:
    """
    Convierte una timeline en intervalos contiguos: [{"start":t0,"end":t1,"slide":id}, ...]
    El último intervalo termina en +inf (None).
    """
    out = []
    for i, (t, slide) in enumerate(timeline):
        t_next = timeline[i+1][0] if i+1 < len(timeline) else None
        out.append({"start": t, "end": t_next, "slide": slide})
    return out

In [10]:
from pathlib import Path
import datasets as ds


def build_dataset(folder="train", language="es", files=None,
                  dpi=150, target_size=(1600,900), mode="letterbox",
                  bg="white", store_slide_paths=False):
    rows = []
    files = get_files(folder) if files is None else files

    # Carga y parsea el maestro (una vez)
    blocks = parse_slide_blocks(f'{folder}.lst')

    for name in files:
        base = Path(f"/content/{folder}/{name}/manual_translations/{language}")
        with open(base / f"{name}.lst") as f:
            times = [(float(a), float(b)) for a,b in (ln.split() for ln in f.read().splitlines())]
        with open(base / f"{name}.en") as f:
            src = f.read().splitlines()
        with open(base / f"{name}.{language}") as f:
            tgt = f.read().splitlines()
        assert len(times) == len(src) == len(tgt)


        # 2) Audio (como ya hacías)
        wav_path = str(m4a_to_wav(f"/content/{folder}/{name}/{name}.m4a"))

        if str(name) in blocks:
            intervals = build_slide_intervals(blocks[str(name)])
        else:
            intervals = [{"start":0.0,"end":None,"slide":None}]
            print(f"[WARN] No hay bloque de slides para {name}")
        if str(name) in DESCRIPTIONS:
            desc = DESCRIPTIONS[str(name)]
        else:
            desc = None
            print(f"[WARN] No hay descripcion para {name}")


        # 3) Crea filas por segmento reusando las mismas referencias a slides
        for (s, e), txt_in, txt_out in zip(times, src, tgt):
            slides  = slides_in_interval(s, e, intervals)
            row = {
                "wav_path": wav_path,
                "start": s,
                "end": e,
                "source_text": txt_in,
                "target_text": txt_out,
                "slide_text": desc[slides[0]] if slides and desc else None ,

            }
            rows.append(row)

    return ds.Dataset.from_list(rows)
test_ds   = build_dataset("test")

['404', '596001', '605000', '606', '545006']


### Load Model

In [11]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

import torch

model_name = 'microsoft/Phi-4-multimodal-instruct'
processor = AutoProcessor.from_pretrained(model_name,trust_remote_code = True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    # if you do not use Ampere or later GPUs, change attention to "eager"
    _attn_implementation='eager',
)

generation_config = GenerationConfig.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

processing_phi4mm.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-multimodal-instruct:
- processing_phi4mm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:594: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/root/.cache/huggingface/modules/transformers_modules/microsoft/Phi-4-multimodal-instruct/33e62acdd07cd7d6635badd529aa0a3467bb9c6a/speech_conformer_encoder.py:2774: FutureWarning: Please specify CheckpointImpl.NO_REENTRANT as CheckpointImpl.REENTRANT will soon be removed as the default and eventually deprecated.
  lambda i: encoder_checkpoint_wrapper(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Basic Inference

In [17]:
def build_prompt_from_slide_texts(slide_text):
    """
    slide_texts: lista de strings (resúmenes de slide).
    """

    if slide_text:
        slide_text=slide_text.strip()

        content = (
            "<|audio_1|>\n"
            "Eres un asistente docente. Usa el siguiente TEXTO DE LA DIAPOSITIVA "
            "solo como CONTEXTO para desambiguar nombres propios, acrónimos y términos técnicos del audio.\n"
            "PRIORIDAD: el contenido hablado del audio.\n\n"
            "REGLAS:\n"
            "- No copies ni parafrasees texto del contexto si no aparece en el audio.\n"
            "- Si hay conflicto, prioriza el audio.\n"
            "- Conserva números, unidades, símbolos y nombres propios.\n"
            "- Si algo no se entiende, escribe [inaudible].\n"
            "[CONTEXTO-DE-DIAPOS]\n"
            f"{slide_text}\n"
            "[/CONTEXTO-DE-DIAPOS]\n\n"
            "Tarea: traduce al español (es-ES) únicamente lo que se dice en el audio.\n"
            "Formato de salida: SOLO la traducción, en una línea, sin etiquetas ni comentarios."
        )
    else :
        content = (
        "<|audio_1|>\n"
        "Eres un asistente docente.\n"
        "Tarea: traduce al español (es-ES) únicamente lo que se dice en el audio."
        )

    return processor.tokenizer.apply_chat_template(
        [{"role":"user","content": content}],
        tokenize=False, add_generation_prompt=True
    )


def load_audio_segment(wav_path, start, end, sr=16_000):
    # lee sólo el fragmento que necesitas
    with sf.SoundFile(wav_path) as f:
        f.seek(int(start*sr))
        audio = f.read(int((end-start)*sr), dtype="float32")
    return audio, sr


In [14]:
test_ds[0]

{'wav_path': '/content/test/404/404.wav',
 'start': 0.0,
 'end': 2.13,
 'source_text': 'Good morning, afternoon and evening.',
 'target_text': 'Buenos días, buenas tardes, buenas noches.',
 'slide_text': '1) Título/tema: La diapositiva se titula "e-sesion 404" y discute el impacto de la cirugía oncológica en los resultados.\n2) 2–3 ideas clave: La diapositiva presenta a dos expertos, Prof. Charles M. Balch y Prof. Riccardo Audisio, que discuten este tema. También menciona que el contenido de la diapositiva se extrajo de la política de e-ESO, que proporciona nuevas habilidades y conocimientos para oncólogos y otros médicos interesados en oncología.\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas.'}

In [15]:
build_prompt_from_slide_texts(test_ds[0]['slide_text'])

'<|user|><|audio_1|>\nEres un asistente docente. Usa el siguiente TEXTO DE LA DIAPOSITIVA solo como CONTEXTO para desambiguar nombres propios, acrónimos y términos técnicos del audio.\nPRIORIDAD: el contenido hablado del audio.\n\nREGLAS:\n- No copies ni parafrasees texto del contexto si no aparece en el audio.\n- Si hay conflicto, prioriza el audio.\n- Conserva números, unidades, símbolos y nombres propios.\n- Si algo no se entiende, escribe [inaudible].\n[CONTEXTO-DE-DIAPOS]\n1) Título/tema: La diapositiva se titula "e-sesion 404" y discute el impacto de la cirugía oncológica en los resultados.\n2) 2–3 ideas clave: La diapositiva presenta a dos expertos, Prof. Charles M. Balch y Prof. Riccardo Audisio, que discuten este tema. También menciona que el contenido de la diapositiva se extrajo de la política de e-ESO, que proporciona nuevas habilidades y conocimientos para oncólogos y otros médicos interesados en oncología.\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmu

In [20]:
# ej.: slide_texts_seg = [resumen_slide_i, resumen_slide_j, ...]  # solo las relevantes
prompt = build_prompt_from_slide_texts(test_ds[0]['slide_text'])
device = torch.device("cuda:0")

wav = test_ds[0]['wav_path']
clips, sr = load_audio_segment(wav, test_ds[0]['start'], test_ds[0]['end'])

inputs = processor(
    text=[prompt],
    audios=[(clips, sr)],
    return_tensors="pt",
    padding=True
).to(device)

model.eval(); model.gradient_checkpointing_disable()
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    gen_ids = model.generate(**inputs, max_new_tokens=256, do_sample=False, num_logits_to_keep=0, use_cache=True)
out = processor.batch_decode(gen_ids[:, inputs["input_ids"].shape[1]:], skip_special_tokens=True)[0]
print(out)  # ← SOLO la traducción


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Buenos días, al mediodía y por la noche.


In [23]:
test_ds[5]

{'wav_path': '/content/test/404/404.wav',
 'start': 17.23,
 'end': 24.87,
 'source_text': 'At the end of the presentation by closing the webcast window you will be directed to the CME evaluation and multiple choice test.',
 'target_text': 'Al final de la presentación, al cerrar la ventana de retransmisión, se le redirigirá a la evaluación de CME y al test de opciones múltiples.',
 'slide_text': '1) Título/tema: La diapositiva se titula "e-sesion 404" y discute el impacto de la cirugía oncológica en los resultados.\n2) 2–3 ideas clave: La diapositiva presenta a dos expertos, Prof. Charles M. Balch y Prof. Riccardo Audisio, que discuten este tema. También menciona que el contenido de la diapositiva se extrajo de la política de e-ESO, que proporciona nuevas habilidades y conocimientos para oncólogos y otros médicos interesados en oncología.\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas.'}

In [24]:
# ej.: slide_texts_seg = [resumen_slide_i, resumen_slide_j, ...]  # solo las relevantes
prompt = build_prompt_from_slide_texts(test_ds[5]['slide_text'])
device = torch.device("cuda:0")

wav = test_ds[5]['wav_path']
clips, sr = load_audio_segment(wav, test_ds[5]['start'], test_ds[5]['end'])

inputs = processor(
    text=[prompt],
    audios=[(clips, sr)],
    return_tensors="pt",
    padding=True
).to(device)

model.eval(); model.gradient_checkpointing_disable()
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    gen_ids = model.generate(**inputs, max_new_tokens=256, do_sample=False, num_logits_to_keep=0, use_cache=True)
out = processor.batch_decode(gen_ids[:, inputs["input_ids"].shape[1]:], skip_special_tokens=True)[0]
print(out)  # ← SOLO la traducción

Al final de la presentación, al cerrar la ventana de transmisión web, se le dirigirá a la evaluación de CME y el examen de opción múltiple.


## Baseline

### Run

In [32]:
import soundfile as sf
from collections import defaultdict

_open_wavs = {}                  # caché: path → SoundFile

def read_segment(path, start, end, sr=16_000):
    """Lee [start,end] s de un WAV grande sin cargarlo entero."""
    if path not in _open_wavs:
        _open_wavs[path] = sf.SoundFile(path)
    f = _open_wavs[path]
    frames = int((end - start) * sr)
    f.seek(int(start * sr))
    return f.read(frames, dtype="float32")

def segments_by_audio(ds):
    groups = defaultdict(list)
    for ex in ds:
        groups[ex["wav_path"]].append(ex)
    # ordenamos cada audio por inicio temporal
    for segs in groups.values():
        segs.sort(key=lambda x: x["start"])
    return groups


def build_prompt_from_slide_texts(slide_text):
    """
    slide_texts: lista de strings (resúmenes de slide).
    use_top_k: si pones un int, usa solo los K primeros (para ahorrar tokens).
    """

    if slide_text:
        slide_texts=slide_text.strip()

        content = (
            "<|audio_1|>\n"
            "Eres un asistente docente. Usa el siguiente TEXTO DE LA DIAPOSITIVA "
            "solo como CONTEXTO para desambiguar nombres propios, acrónimos y términos técnicos del audio.\n"
            "PRIORIDAD: el contenido hablado del audio.\n\n"
            "REGLAS:\n"
            "- No copies ni parafrasees texto del contexto si no aparece en el audio.\n"
            "- Si hay conflicto, prioriza el audio.\n"
            "- Conserva números, unidades, símbolos y nombres propios.\n"
            "- Si algo no se entiende, escribe [inaudible].\n"
            "- No incluyas simbolos extra\n"
            "[CONTEXTO-DE-DIAPOS]\n"
            f"{slide_text}\n"
            "[/CONTEXTO-DE-DIAPOS]\n\n"
            "Tarea: traduce al español (es-ES) únicamente lo que se dice en el audio.\n"
            "Formato de salida: SOLO la traducción, en una línea, sin etiquetas ni comentarios."
        )
    else :
        content = (
        "<|audio_1|>\n"
        "Eres un asistente docente."
        "Tarea: traduce al español (es-ES) únicamente lo que se dice en el audio."
        )

    return processor.tokenizer.apply_chat_template(
        [{"role":"user","content": content}],
        tokenize=False, add_generation_prompt=True
    )


In [26]:
groups = segments_by_audio(test_ds)
groups.keys()

dict_keys(['/content/test/404/404.wav', '/content/test/596001/596001.wav', '/content/test/605000/605000.wav', '/content/test/606/606.wav', '/content/test/545006/545006.wav'])

In [31]:
groups.keys()

dict_keys(['/content/test/596001/596001.wav', '/content/test/605000/605000.wav', '/content/test/606/606.wav', '/content/test/545006/545006.wav'])

In [ ]:

import torch, gc
from tqdm import tqdm

device        = torch.device("cuda:0")
batch_size    = 8


def generate_output(groups):
    hyps_all, refs_all, srcs_all = [], [], []

    for wav_path, segs in tqdm(groups.items(), desc="Audios"):
        hyps, refs, srcs = [], [], []

        # dividimos en lotes (batch_size) para GPU
        for i in range(0, len(segs), batch_size):
            chunk = segs[i:i+batch_size]

            wavs   = [(read_segment(s["wav_path"], s["start"], s["end"]),16_000) for s in chunk]
            prompts= [build_prompt_from_slide_texts(s["slide_text"]) for s in chunk]


            inputs = processor(text=prompts, audios=wavs,
                               return_tensors="pt", padding=True
                               ).to(device)

            with torch.inference_mode(), torch.autocast("cuda"):
                gen_ids = model.generate(**inputs, generation_config=generation_config, max_new_tokens=1000, num_logits_to_keep=0)

            # quitamos el prompt textual
            gen_ids = gen_ids[:, inputs["input_ids"].shape[1]:]

            hyps.extend(
                processor.batch_decode(gen_ids.cpu(),
                                       skip_special_tokens=True,
                                       clean_up_tokenization_spaces=False)
            )

            # limpieza VRAM
            del inputs, gen_ids, wavs, prompts
            torch.cuda.empty_cache(); gc.collect()

            refs.extend([s["target_text"] for s in chunk])
            srcs.extend([s["source_text"] for s in chunk])

        print(hyps)
        print(refs)
        print(srcs)


        hyps_all.append(hyps)
        refs_all.append(refs)
        srcs_all.append(srcs)

    return hyps_all, refs_all, srcs_all

hypotesis, traducciones, transcriptions = generate_output(groups)

Audios:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Audios:  25%|██▌       | 1/4 [1:15:42<3:47:08, 4542.95s/it]

['!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Audios:  50%|█████     | 2/4 [3:00:13<3:05:18, 5559.08s/it]

['!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Audios:  75%|███████▌  | 3/4 [3:58:00<1:16:43, 4603.65s/it]

['!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Audios: 100%|██████████| 4/4 [4:30:32<00:00, 4058.05s/it]

['!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [36]:
for h,t, trad in zip(hypotesis, transcriptions, traducciones):
  print(h)
  print(t)
  print(trad)
  print()

['!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

### Evaluation

In [37]:
from __future__ import annotations

from itertools import chain
from typing import Callable, Dict, List, Tuple

import sacrebleu
from comet import download_model, load_from_checkpoint

In [38]:


# ────────────────────────────────────────────────────────────────────────────────
#  Cargamos (y cacheamos) el modelo COMET‑22 una sola vez
# ────────────────────────────────────────────────────────────────────────────────
_COMET_MODEL = None
_COMET_PATH = None
_MODEL_NAME = "Unbabel/wmt22-comet-da"


def _get_comet_model(gpus: int = 0):
    """Devuelve un modelo COMET‑22 listo para `predict` (cacheado)."""
    global _COMET_MODEL, _COMET_PATH

    if _COMET_MODEL is None:
        _COMET_PATH = download_model(_MODEL_NAME)  # se guarda en ~/.cache
        _COMET_MODEL = load_from_checkpoint(_COMET_PATH)

    # *Nota*: el parámetro `gpus` se pasa a `.predict` y **no** aquí, pero
    # exponemos el arg para quien quiera forzar CPU en la firma pública.
    return _COMET_MODEL


# ────────────────────────────────────────────────────────────────────────────────
#  Función principal
# ────────────────────────────────────────────────────────────────────────────────

def bleu_comet_by_audio(
    refs_audio: List[List[str]],
    hyps_audio: List[List[str]],
    srcs_audio: List[List[str]],
    transform: Callable[[str], str] = lambda x: x,
    comet_gpus: int = 0,
    comet_batch_size: int = 8,
) -> Tuple[Dict[str, float], List[Dict[str, float]]]:
    """Calcula BLEU y COMET‑22, **global** y **por audio**.

    Parameters
    ----------
    refs_audio, hyps_audio, srcs_audio : list[list[str]]
        Listas anidadas con el mismo nº de audios y segmentos.
    transform : callable
        Función de normalización por frase (identidad por defecto).
    comet_gpus : int
        Nº de GPUs a usar en `model.predict` (0 ⇒ CPU).
    comet_batch_size : int
        Tamaño de lote para COMET (trade‑off velocidad / memoria).

    Returns
    -------
    (global_metrics, per_audio_metrics)
        global_metrics    = {"bleu": float, "comet22": float}
        per_audio_metrics = [
            {"audio_id": i, "bleu": float, "comet22": float},
            ...
        ]
    """

    # ── Comprobaciones básicas ──────────────────────────────────────────────
    assert len(refs_audio) == len(hyps_audio) == len(srcs_audio), (
        "refs, hyps y srcs deben tener la misma longitud (nº audios)"
    )

    per_audio: List[Dict[str, float]] = []

    # ── Recorremos audio por audio ──────────────────────────────────────────
    comet_model = _get_comet_model()

    for idx, (ref_seg, hyp_seg, src_seg) in enumerate(
        zip(refs_audio, hyps_audio, srcs_audio)
    ):
        assert len(ref_seg) == len(hyp_seg) == len(src_seg), (
            f"El audio {idx} contiene diferente nº de segmentos"
        )

        ref_seg = [transform(r) for r in ref_seg]
        hyp_seg = [transform(h) for h in hyp_seg]
        src_seg = [transform(s) for s in src_seg]

        # ── BLEU corpus‑level para el audio ────────────────────────────────
        bleu_score = sacrebleu.corpus_bleu(hyp_seg, [ref_seg]).score

        # ── COMET‑22 ───────────────────────────────────────────────────────
        samples = [  # una entrada por segmento
            {"src": s, "mt": h, "ref": r}
            for s, h, r in zip(src_seg, hyp_seg, ref_seg)
        ]
        comet_out = comet_model.predict(
            samples,
            batch_size=comet_batch_size,
            gpus=comet_gpus,
            progress_bar=False,
        )
        comet_score = comet_out["system_score"]  # media ya calculada

        per_audio.append(
            {
                "audio_id": idx,
                "bleu": bleu_score,
                "comet22": comet_score,
            }
        )

    # ── Métricas globales ──────────────────────────────────────────────────
    refs_all = list(chain.from_iterable(refs_audio))
    hyps_all = list(chain.from_iterable(hyps_audio))
    srcs_all = list(chain.from_iterable(srcs_audio))

    bleu_global = sacrebleu.corpus_bleu(hyps_all, [refs_all]).score

    comet_samples = [
        {"src": s, "mt": h, "ref": r}
        for s, h, r in zip(srcs_all, hyps_all, refs_all)
    ]
    comet_global = comet_model.predict(
        comet_samples,
        batch_size=comet_batch_size,
        gpus=comet_gpus,
        progress_bar=False,
    )["system_score"]

    return {"bleu": bleu_global, "comet22": comet_global}, per_audio





### Results

In [39]:
global_m, per_audio_m = bleu_comet_by_audio(traducciones, hypotesis, transcriptions, comet_gpus=1)

print("── Métricas globales ──")
for k, v in global_m.items():
    print(f"{k:8s}: {v:.2f}")

print("\n── Métricas por audio ──")
for m in per_audio_m:
    print(
        f"Audio {m['audio_id']}: BLEU={m['bleu']:.2f}, COMET-22={m['comet22']:.2f}"
    )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

checkpoints/model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/sta

── Métricas globales ──
bleu    : 0.17
comet22 : 0.22

── Métricas por audio ──
Audio 0: BLEU=0.17, COMET-22=0.21
Audio 1: BLEU=0.14, COMET-22=0.21
Audio 2: BLEU=0.16, COMET-22=0.21
Audio 3: BLEU=0.10, COMET-22=0.20
Audio 4: BLEU=0.37, COMET-22=0.26
